In [1]:
"""This script converts extraHAWC data to FITS format."""

'This script converts extraHAWC data to FITS format.'

In [3]:
import numpy as np
import pickle

from feupy.config import *
from feupy.utils.string_handling import *
from feupy.utils.table import *

from astropy.coordinates import SkyCoord
from astropy.table import Table, Column

from gammapy.utils.scripts import make_path
from gammapy.modeling.models import LogParabolaSpectralModel, ExpCutoffPowerLawSpectralModel, PowerLawSpectralModel

In [ ]:
data = {
    "2HWC J1837-065": {
        "position": SkyCoord(279.36, -6.58, unit=UNIT_DEG, frame=FRAME_ICRS), 
        'spectral_model': PowerLawSpectralModel(
            index=2.90,
            amplitude="85.2e-15 cm-2 s-1 TeV-1",
            reference=7 * u.TeV,),
        'reference': '2HWC catalogue',},
    "2HWC J1825-134": {
        "position":  SkyCoord(276.46, -13.40, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': PowerLawSpectralModel(
            index=2.58,
            amplitude="138.0e-15 cm-2 s-1 TeV-1",
            reference=7 * u.TeV,),
        'reference': '2HWC catalogue',}, 
    "HAWC J1825-138": {
        "position":  SkyCoord(276.38, -13.86, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': ExpCutoffPowerLawSpectralModel(
            amplitude=2.7e-14 * u.Unit("cm-2 s-1 TeV-1"),
            index=2.02,
            lambda_=1/27 * u.Unit("TeV-1"),
            reference=18 * u.TeV,),
        'reference': 'https://arxiv.org/pdf/2012.15275.pdf',},
    "HAWC J1826-128": {
        "position":  SkyCoord(276.50, -12.86, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': ExpCutoffPowerLawSpectralModel(
            amplitude=2.7e-14 * u.Unit("cm-2 s-1 TeV-1"),
            index=1.2,
            lambda_=(1/24) * u.Unit("TeV-1"),
            reference=18 * u.TeV,),
        'reference': 'https://arxiv.org/pdf/2012.15275.pdf',},
    "HAWC J1825-134": {
        "position":  SkyCoord(276.44, -13.42, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': PowerLawSpectralModel(
            index=2.28,
            amplitude="4.2e-15 TeV-1 cm-2 s-1",
            reference=18 * u.TeV,),
        'reference': 'https://arxiv.org/pdf/2012.15275.pdf',},
    "eHWC J1825-134": {
        "position":  SkyCoord(276.40, -13.37, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': ExpCutoffPowerLawSpectralModel(
            amplitude=2.12e-13 * u.Unit("cm-2 s-1 TeV-1"),
            index=2.12,
            lambda_= (1/61) * u.Unit("TeV-1"),
            reference=10 * u.TeV,),
        'reference': 'https://arxiv.org/pdf/1909.08609.pdf',},
    "eHWC J1907+063": {
        "position":  SkyCoord(286.91, 6.32, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': LogParabolaSpectralModel(
             alpha=2.46,
            amplitude="0.95e-13 cm-2 s-1 TeV-1",
            reference=10 * u.TeV,
            beta=0.11,),
        'reference': 'https://arxiv.org/pdf/1909.08609.pdf',}, 
    "eHWC J2019+368": {
        "position":  SkyCoord(304.95, 36.78, unit=UNIT_DEG, frame=FRAME_ICRS),
        'spectral_model': LogParabolaSpectralModel(
            alpha=2.08,
            amplitude="0.45e-13 cm-2 s-1 TeV-1",
            reference=10 * u.TeV,
            beta=0.26,),
        'reference': 'https://arxiv.org/pdf/1909.08609.pdf',}, 
}
data_name='extraHAWC'

In [ ]:
# with open(f"{data_name}.pkl", "wb") as fp:  
#         pickle.dump(data, fp)     
# with open(f"{data_name}.pkl", "rb") as fp:  
#     data = pickle.load(fp)
# data

In [ ]:
names = list(data.keys())
array = np.empty((len(names)))

table = Table()
table.meta['catalog_name'] = data_name
table.meta["SED_TYPE"] = "e2dnde"
table.meta['reference'] = '...'
table["index"] = list(range(0, len(names), 1))
table['source_name'] = names
table['source_name'].description = 'Source name'
table['ra'] = Column(
    data=array,
    description='Right Ascension (J2000)', unit='deg', format='.2f',
)
table['dec'] = Column(
    data=array,
    description='Declination (J2000)', unit='deg', format='.2f',
)

table['spectral_model_type'] = names
table['spectral_model_type'].description = 'Spectral Model Type'

# table['spectral_model'].dtype = SpectralModel

for index, name in enumerate(names):
    table['ra'][index] =  data[name]["position"].ra.value
    table['dec'][index] = data[name]["position"].dec.value
    if data[name]["spectral_model"]:
        table['spectral_model_type'][index] = data[name]["spectral_model"].tag[1]
    else: table['spectral_model_type'][index] = None

In [ ]:
# table.write(f"{data_name}.fits",
#     format = 'fits', 
#     overwrite = True
# ) 
# table.read(f"{data_name}.fits")   

In [ ]:
FILE_PATH = "$PYTHONPATH/feupy/data/catalogs/extraHAWC/flux_points_tables/"
length=10
e_ref = []
e2dnde = []
e2dnde_errp = []
e2dnde_errn = []
is_ul = []
for index, name in enumerate(names):
    _table = Table.read(make_path(f'{FILE_PATH}{name_to_txt(name)}.fits')) 
    e_ref.append(append_nones(length=length, list_=list(_table['e_ref'])))
    e2dnde.append(append_nones(length=length, list_=list(_table['e2dnde'])))
    e2dnde_errp.append(append_nones(length=length, list_=list(_table['e2dnde_errp'])))
    e2dnde_errn.append(append_nones(length=length, list_=list(_table['e2dnde_errn'])))
    is_ul.append(append_nones(length=length, list_=list(_table['is_ul'])))
col_1 = Column(name='e_ref', description='Reference Energy', data=u.Quantity(e_ref, 'TeV'))
col_2 = Column(name='e2dnde', description='e2dnde', data=u.Quantity(e2dnde, 'TeV cm-2 s-1'))
col_3 = Column(name='e2dnde_errp', description='e2dnde_errp', data=u.Quantity(e2dnde_errp, 'TeV cm-2 s-1'))
col_4 = Column(name='e2dnde_errn', description='e2dnde_errn', data=u.Quantity(e2dnde_errn, 'TeV cm-2 s-1'))
col_5 = Column(name='is_ul', description='is_ul', data=is_ul,dtype=bool)
table.add_columns([col_1,col_2,col_3,col_4,col_5])
# table

In [ ]:
table.write(f"{data_name}.fits",
    format = 'fits', 
    overwrite = True
) 